In [1]:
from code.neuron import *
from code.encodingsource import *
from code.hsgs import *
from code.classical_neuron import *
from code.classical_pso import *
from code.sf import *
simulator = Aer.get_backend('qasm_simulator')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import time 

from code.experiment_functions import * 

In [2]:
X_train = [[-1, -1],  [-1,1], [1,1], [1,-1]]
y_train = [1, 0, 1, 0]

X_test =  [[-1, -1],  [-1,1], [1,1], [1,-1]]
y_test =  [1, 0, 1, 0]

In [3]:
with open('results/version7/train_xor.json', 'w+') as fp:
    json.dump([X_train, y_train], fp)
    
with open('results/version7/test_xor.json', 'w+') as fp:
    json.dump([X_test, y_test], fp)

## Get Error by Epoch

In [4]:
epochs = {'model':[], 'epoch':[], 'value':[]}

In [5]:
initial_weights = list(np.random.uniform(low=0.0, high=1.0, size=(len(X_train[0]),)))

In [6]:
weights = quantumNeuronFIT(X_train, 
                           y_train,
                           init_weight = initial_weights,
                           threshold=0.5,
                           lrParameter=0.02, 
                           n_epochs=60, 
                           trainingBias=True,
                           epoch_results=False,
                           trainingApproaches={'hsgs'})

lrParameter:  0.02
threshold:  0.5
trainingBias:  True
phaseEstrategyOperator:  phase-encoding-phase
best error HSGS training:  1


In [7]:
epochs['value'].extend(weights[4])
epochs['model'].extend(['BQN' for i in range(len(weights[4]))])
epochs['epoch'].extend(list(range(len(weights[4]))))

In [8]:
for strategy in ['original', 'angleradius']:
    weights = quantumNeuronFIT(X_train, 
                           y_train,
                           init_weight=initial_weights,
                           threshold=0.8,
                           lrParameter=0.02, 
                           n_epochs=50, 
                           trainingBias=True,
                           phaseEstrategyOperator=strategy,
                           trainingApproaches={'phase-encoding'})
    
    epochs['value'].extend(weights[5])
    epochs['model'].extend([strategy for i in range(len(weights[5]))])
    epochs['epoch'].extend(list(range(len(weights[5]))))

lrParameter:  0.02
threshold:  0.8
trainingBias:  True
phaseEstrategyOperator:  original

best error phase-encoding training:  1
lrParameter:  0.02
threshold:  0.8
trainingBias:  True
phaseEstrategyOperator:  angleradius

best error phase-encoding training:  0


In [9]:
for strategy in ['angle', 'radius']:
    weights = quantumNeuronFIT(X_train, 
                           y_train,
                           init_weight=initial_weights,
                           threshold=0.5,
                           lrParameter=0.02, 
                           n_epochs=60, 
                           trainingBias=False,
                           phaseEstrategyOperator=strategy,
                           trainingApproaches={'phase-encoding'})
    
    epochs['value'].extend(weights[5])
    epochs['model'].extend([strategy for i in range(len(weights[5]))])
    epochs['epoch'].extend(list(range(len(weights[5]))))

lrParameter:  0.02
threshold:  0.5
trainingBias:  False
phaseEstrategyOperator:  angle

best error phase-encoding training:  1
lrParameter:  0.02
threshold:  0.5
trainingBias:  False
phaseEstrategyOperator:  radius

best error phase-encoding training:  2


In [10]:
pd.DataFrame(epochs).to_csv('results/version7/error_by_epoch_bias.csv')

## Continuously Valued Quantum Neuron Experiments

In [ ]:
# search space
initial_weights =  list(np.random.uniform(low=0.0, high=1.0, size=(len(X_test[0]),)))
threshold_space = [0.1, 0.3, 0.5, 0.7, 0.8]
lr_space = [0.02, 0.1]
bias_space = [False, True]
strategy_space = ['original', 'angle', 'radius', 'angleradius']

In [ ]:
%%time
for w in initial_weights:
    for strategy in strategy_space:
        for threshold in threshold_space:
            for lr in lr_space:
                for bias in bias_space:
                    # execute experiment
                    weights = quantumNeuronFIT(X_train, 
                                               y_train,
                                               init_weight=initial_weights,
                                               threshold=threshold,
                                               lrParameter=lr, 
                                               n_epochs=20, 
                                               trainingBias=bias,
                                               phaseEstrategyOperator=strategy,
                                               trainingApproaches={'phase-encoding'})

                    results = quantumNeuronPREDICT(X_test, 
                                                       y_test,
                                                       threshold=threshold,
                                                       weightVectorsPhaseEncoding = weights[0], 
                                                       weightVectorsHSGS = weights[1], 
                                                       repeat=5,
                                                       bias=bias, 
                                                       phaseEstrategyOperator=strategy,
                                                       testingApproaches={'phase-encoding'})

                    # get experiment results
                    output['model'].append('phase-encoding')
                    output['lr'].append(lr)
                    output['bias'].append(bias)
                    output['threshold'].append(threshold)
                    output['phase_strategy'].append(strategy)
                    output['avg_error'].append(results[0])
                    output['neuron_outputs'].append(results[1])
                    output['trained_weights'].append(results[2])
                    output['initial_weights'].append(w)
                    print('\n')

In [ ]:
experiment_phase = pd.DataFrame(output)
experiment_phase.to_csv('results/version7/experiment_XOR_phase.csv')